In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
#LangSmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")


In [11]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_groq import ChatGroq
llm=ChatGroq(
    model_name="llama-3.3-70b-versatile"
)


In [12]:
##Data Ingestion-From the website we need to scrap the data
import requests
from langchain_community.document_loaders import WebBaseLoader

In [13]:
loader=WebBaseLoader("https://docs.langchain.com/langsmith/optimize-classifier")

In [14]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/optimize-classifier', 'title': 'Optimize a classifier - Docs by LangChain', 'language': 'en'}, page_content='Optimize a classifier - Docs by LangChainSkip to main contentDocs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationTutorialsOptimize a classifierGet startedObservabilityEvaluationPrompt engineeringDeploymentPlatform setupReferenceOverviewQuickstartConceptsCreate and update promptsCreate a promptManage promptsManage prompts programmaticallyPrompt template formatConfigure prompt settingsUse tools in a promptInclude multimodal content in a promptWrite your prompt with AIConnect to modelsTutorialsOptimize a classifierSync prompts with GitHubTest multi-turn conversationsOn this pageThe objectiveGetting startedSet up automationsUpdate the applicationSemantic search over examplesTutorialsOptimize a classifierCopy pageCopy pageThis tutorial walks through optimizing a cla

In [15]:
#Dividing into chunks 
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

In [16]:
documents=text_splitter.split_documents(docs)

In [17]:
documents

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/optimize-classifier', 'title': 'Optimize a classifier - Docs by LangChain', 'language': 'en'}, page_content='Optimize a classifier - Docs by LangChainSkip to main contentDocs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationTutorialsOptimize a classifierGet startedObservabilityEvaluationPrompt engineeringDeploymentPlatform setupReferenceOverviewQuickstartConceptsCreate and update promptsCreate a promptManage promptsManage prompts programmaticallyPrompt template formatConfigure prompt settingsUse tools in a promptInclude multimodal content in a promptWrite your prompt with AIConnect to modelsTutorialsOptimize a classifierSync prompts with GitHubTest multi-turn conversationsOn this pageThe objectiveGetting startedSet up automationsUpdate the applicationSemantic search over examplesTutorialsOptimize a classifierCopy pageCopy pageThis tutorial walks through optimizing a cla

In [21]:
#Converting this text to vectors
embeddings = OllamaEmbeddings(model="mxbai-embed-large")

In [22]:
#We will use FAISS Database
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [23]:
vectorstoredb